### Load libraries and data

In [1]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
plt.style.use(['dark_background'])
%matplotlib inline
import galsim
import pandas as pd
from SpecklePSF import SpeckleSeries as speckle

basedir = '/Users/clairealice/Documents/Research/Burchat/temporal_PSF/speckleImagers/'
cmap = 'plasma'
p_scale = 0.01
gains = {'025':{'a':14.9,'b':12.8},'234':{'a':11.21,'b':11.21},
        '258':{'a':11.21,'b':11.21},'484':{'a':14.9 ,'b':12.8},
        '663':{'a':2.56,'b':11.21},'693':{'a':12.8,'b':12.8},
        '809':{'a':11.21,'b':12.8},'1039':{'a':2.03,'b':2.56},
#         '1237':{'a':32.96,'b':265.83},
        '1262':{'a':11.21,'b':11.21}}
backgrounds = {'025':{'a':30.85,'b':53.57},'234':{'a':42.74,'b':139.74},
               '258':{'a':69.39,'b':144.92},'484':{'a':33.65 ,'b':61.42},
               '663':{'a':32.13,'b':88.25},'693':{'a':73.12,'b':81.39},
               '809':{'a':39.08,'b':59.07},'1039':{'a':26.04,'b':99.80},
#                '1237':{'a':32.96,'b':265.83},
               '1262':{'a':84.10,'b':140.63}}

ModuleNotFoundError: No module named 'SpecklePSF'

### Load files

In [ ]:
filenumber = '025'
sample_index = 6

filename = 'img_a_{}.fits'.format(filenumber)
hdulist = fits.open(basedir + filename)
dssi_a = hdulist[0].data
a_gain = gains[filenumber]['a']
sample = dssi_a[sample_index]
a_std_back = backgrounds[filenumber]['a']

filename = 'img_b_{}_cumulative.fits'.format(filenumber)
hdulist = fits.open(basedir + filename)
dssi_b = hdulist[0].data
b_gain = gains[filenumber]['b']
bsample = np.fliplr(dssi_b[sample_index])
b_std_back = backgrounds[filenumber]['b']

# Estimating backgrounds

In [ ]:
#example 
std = estimate_back(dssi_a[10],20)
print "standard dev: ",std

In [ ]:
def estimate_dataset_backgrounds(dssi_a,dssi_b,filenumber):
    std_back_a = np.zeros(20)
    std_back_b = np.zeros(20)

    pts = np.linspace(0,950,20)
    for i in pts:
        if (filenumber == '234') & (int(i) == 500):
            i = 501
        std_back_a[int(i/50)] = estimate_back(dssi_a[int(i)],20,plot=False)
        std_back_b[int(i/50)] = estimate_back(dssi_b[int(i)],20,plot=False)

    plt.figure(figsize=(8,5))
    plt.plot(pts,std_back_b,'ro',label='880nm; average %.2f'%np.mean(std_back_b))
    plt.plot(pts,std_back_a,'bo',label='692nm; average %.2f'%np.mean(std_back_a))
    plt.xlabel('frame index',fontsize=12)
    plt.ylabel('background $\sigma$',fontsize=12)
    plt.ylim(ymin=0)
    plt.legend(loc=8)
    plt.savefig(basedir+'backgrounds/2sigma_clipped_background_sqrmask_{}.png'.format(filenumber))
    #plt.show()

In [ ]:
for filenumber in backgrounds.keys():
    filename = 'img_a_{}.fits'.format(filenumber)
    hdulist = fits.open(basedir + filename)
    dssi_a = hdulist[0].data*gains[filenumber]['a']

    filename = 'img_b_{}.fits'.format(filenumber)
    hdulist = fits.open(basedir + filename)
    dssi_b = hdulist[0].data*gains[filenumber]['b']
    
    estimate_dataset_backgrounds(dssi_a,dssi_b,filenumber)

## Fit single sample

### filter a = 692nm

In [ ]:
result = fit_single_exposure(sample*a_gain,p_scale,a_std_back)
result_P = result.params
plot_datamodelres(sample,result_P,a_gain)

In [ ]:
print 'chi squared:', '%.3f' %result.chisqr
print 'size (hlr) :', '%.3f' % result_P['hlr']
print 'g1 :', '%.4f' % result_P['g1']
print 'g2 :', '%.4f' % result_P['g2']
print 'x offset :', '%.4f' % result_P['offset_x']
print 'y offset :', '%.4f' % result_P['offset_y']
print 'flux :', '%.4f' % result_P['flux']
print 'background :', '%.4f' % result_P['background']

### filter b = 880nm

In [ ]:
bresult = fit_single_exposure(bsample*b_gain,p_scale,b_std_back,N_exp=12)
bresult_P = bresult.params
plot_datamodelres(bsample,bresult_P,b_gain)

In [ ]:
print 'chi squared :', '%.3f' % bresult.chisqr
print 'size (hlr) :', '%.3f' % bresult_P['hlr']
print 'g1 :', '%.4f' % bresult_P['g1']
print 'g2 :', '%.4f' % bresult_P['g2']
print 'x offset :', '%.4f' % bresult_P['offset_x']
print 'y offset :', '%.4f' % bresult_P['offset_y']
print 'flux :', '%.4f' % bresult_P['flux']
print 'background :', '%.4f' % bresult_P['background']

# Residuals

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
plt.hist(sample-model, bins = np.linspace(np.min(sample-model),np.max(sample-model),100))

plt.subplot(1,2,2)
plt.hist(bsample-bmodel, bins = np.linspace(np.min(bsample-bmodel),np.max(bsample-bmodel),100))

plt.show()

### Slice through image

In [ ]:
model = plot_k_model(result_P,np.shape(sample))
bmodel = plot_k_model(bresult_P,np.shape(bsample))

x = int(result_P['offset_x'])+128
y = int(result_P['offset_y'])+128
bx = int(bresult_P['offset_x'])+128
by = int(bresult_P['offset_y'])+128

s_a_slice_x = sample[x]*a_gain
m_a_slice_x = model[x]
s_b_slice_x = bsample[bx]*b_gain
m_b_slice_x = bmodel[bx]

s_a_slice_y = sample[:,y]*a_gain
m_a_slice_y = model[:,y]
s_b_slice_y = bsample[:,by]*b_gain
m_b_slice_y = bmodel[:,by]

a_err_x = np.sqrt(a_std_back**2 + abs(s_a_slice_x-result_P['background']))
b_err_x = np.sqrt(b_std_back**2 + abs(s_b_slice_x-bresult_P['background']))
a_err_y = np.sqrt(a_std_back**2 + abs(s_a_slice_y-result_P['background']))
b_err_y = np.sqrt(b_std_back**2 + abs(s_b_slice_y-bresult_P['background']))

plt.figure(figsize=(14,16))

plt.subplot(4,2,1)
# plt.plot(s_a_slice_x,'k',ls='steps',label='data')
plt.fill_between(np.linspace(0,256,256),s_a_slice_x+a_err_x,s_a_slice_x-a_err_x,facecolor='b',\
                 edgecolor='darkblue',alpha=0.75,label='data')
plt.plot(np.linspace(0,256,256),m_a_slice_x,'k',label='model')
plt.ylabel('pixel intensity',fontsize=12)
plt.xlim([0,256])
plt.title('filter a: x cross section',fontsize=14)
plt.legend()

plt.subplot(4,2,3)
# plt.plot(s_a_slice_y,'k',ls='steps',label='data')
plt.fill_between(np.linspace(0,256,256),s_a_slice_y+a_err_y,s_a_slice_y-a_err_y,facecolor='b',\
                 edgecolor='darkblue',alpha=0.75,label='data')
plt.plot(np.linspace(0,256,256),m_a_slice_y,'k',label='model')
plt.ylabel('pixel intensity',fontsize=12)
plt.xlim([0,256])
plt.title('filter a: y cross section',fontsize=14)
plt.legend()

plt.subplot(4,2,5)
# plt.plot(s_b_slice_x,'k',ls='steps',label='data')
plt.fill_between(np.linspace(0,256,256),s_b_slice_x+b_err_x,s_b_slice_x-b_err_x,facecolor='r',\
                 edgecolor='darkred',alpha=0.75,label='data')
plt.plot(np.linspace(0,256,256),m_b_slice_x,'k',label='model')
plt.ylabel('pixel intensity',fontsize=12)
plt.xlim([0,256])
plt.title('filter b: x cross section',fontsize=14)
plt.legend()

plt.subplot(4,2,7)
# plt.plot(s_a_slice_y,'k',ls='steps',label='data')
plt.fill_between(np.linspace(0,256,256),s_b_slice_y+b_err_y,s_b_slice_y-b_err_y,facecolor='r',\
                 edgecolor='darkred',alpha=0.75,label='data')
plt.plot(np.linspace(0,256,256),m_b_slice_y,'k',label='model')
plt.ylabel('pixel intensity',fontsize=12)
plt.xlim([0,256])
plt.title('filter b: y cross section',fontsize=14)
plt.legend()

plt.subplot(4,2,2)
diff = s_a_slice_x-m_a_slice_x
plt.hist(diff,bins=np.linspace(min(diff),max(diff),100))
plt.title('x slice residuals',fontsize=14)

plt.subplot(4,2,4)
diff = s_a_slice_y-m_a_slice_y
plt.hist(diff,bins=np.linspace(min(diff),max(diff),100))
plt.title('y slice residuals',fontsize=14)

plt.subplot(4,2,6)
diff = s_b_slice_x-m_b_slice_x
plt.hist(diff,bins=np.linspace(min(diff),max(diff),100),color='r')
plt.title('x slice residuals',fontsize=14)

plt.subplot(4,2,8)
diff = s_b_slice_y-m_b_slice_y
plt.hist(diff,bins=np.linspace(min(diff),max(diff),100),color='r')
plt.title('y slice residuals',fontsize=14)

# plt.subplot(3,2,5)
# plt.hist(a_err_x,bins=np.linspace(min(a_err_x),max(a_err_x),50))
# plt.title('estimated errors: background and signal',fontsize=14)
# plt.xlabel('$\sigma$',fontsize=16)

# plt.subplot(3,2,6)
# plt.hist(b_err_x,bins=np.linspace(min(b_err_x),max(b_err_x),50))
# plt.title('estimated errors: background and signal',fontsize=14)
# plt.xlabel('$\sigma$',fontsize=16)

plt.show()

# Binning image

In [ ]:
interm = (sample[:-1]+sample[1:])[::2]
binned_sample = (interm[:,:-1]+interm[:,1:])[:,::2]/4

In [ ]:
binned_result = fit_single_exposure(binned_sample,p_scale=.01*2,a_std_back)
binned_result_P = binned_result.params
binned_model = plot_k_model(binned_result_P,np.shape(binned_sample))

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(2, 2, 1)
plt.imshow(sample, origin='lower', interpolation='nearest',cmap=cmap)
plt.title('data')
plt.colorbar()

plt.subplot(2, 2, 3)
plt.imshow(model,origin='lower',cmap=cmap,vmax=np.max(sample))
plt.title('model')
plt.colorbar()

plt.subplot(2, 2, 2)
plt.imshow(binned_sample, origin='lower', interpolation='nearest',cmap=cmap)
plt.title('binned data')
plt.colorbar()

plt.subplot(2, 2, 4)
plt.imshow(binned_model,origin='lower',cmap=cmap,vmax=np.max(binned_sample))
plt.title('binned model')
plt.colorbar()

plt.show()

In [ ]:
res_table(result_P, binned_result_P, columns=['data','binned data','change'])

# Centering

In [ ]:
centered_sample, g1, g2 = center_psf(sample,p_scale=0.01)

centered_result = fit_single_exposure(centered_sample,p_scale=.01,a_std_back)
centered_result = centered_result.params
centered_model = plot_k_model(centered_result_P,np.shape(centered_sample))

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(2, 2, 1)
plt.imshow(sample, origin='lower', interpolation='nearest',cmap=cmap)
plt.title('data')
plt.colorbar()

plt.subplot(2, 2, 3)
plt.imshow(model,origin='lower',cmap=cmap,vmax=np.max(sample))
plt.title('model')
plt.colorbar()

plt.subplot(2, 2, 2)
plt.imshow(centered_sample, origin='lower', interpolation='nearest',cmap=cmap)
plt.title('centered data')
plt.colorbar()

plt.subplot(2, 2, 4)
plt.imshow(centered_model,origin='lower',cmap=cmap,vmax=np.max(centered_sample))
plt.title('centered model')
plt.colorbar()

plt.show()

In [ ]:
res_table(result_P, centered_result_P, columns=['data','centered data','change'])

### Change of ellipticity with stacking

In [ ]:
N = 10
test=dssi_a_050
stacks = []
for n in range(1,N+1):
    stack = [np.sum(test[k*int(1000/n):(k+1)*int(1000/n)],axis=0) for k in range(n)]
    stacks.append(np.array(stack)*n/1000)
print np.shape(stacks)
print np.shape(stacks[1])

In [ ]:
centered_stacks = []
for series in stacks:
    centered_stack = []
    for i in range(len(series)):
        centered_stack.append( center_psf(series[i],p_scale=0.01)[0] )
    centered_stacks.append(centered_stack)


In [ ]:
##need to trim and stack the centered psfs

imgShapes = [[np.shape(i) for i in j] for j in centered_stacks].flatten()

ymin, xmin = np.min(imgShapes, axis=0)
newShape = (ymin,xmin)

shape_deltas = [(a-ymin,b-xmin) for (a,b) in imgShapes]

final_stacks = np.zeros((N,ymin,xmin))
tc_added = np.zeros((N,ymin,xmin))
for i in range(N):
    (dy,dx) = shape_deltas[i]
    trimmed_centered_imgs[i] = centered_imgs[i][dy/2:imgShapes[i][0]-dy/2,dx/2:imgShapes[i][1]-dx/2]
    tc_added[i] = np.sum(trimmed_centered_imgs,axis=0)

    ## refit and compute difference in fit from 
    result_P_2 = fit_single_exposure(trimmed_centered_imgs[i],p_scale,a_std_back)
    print result_P_2['g1']
    deltas[:,i] -= [(deltas[0,i] - result_P_2['g1']),(deltas[1,i] - result_P_2['g2'])]

In [ ]:
## trial run of just getting ellipticities of centered sub stacks
stacked_ellipticities = []
for series in stacks:
    series_e = np.zeros(len(series))
    for i in range(len(series)):
        centered_img, g1, g2 = center_psf(series[i],p_scale=0.01)

        centered_result_P = fit_single_exposure(centered_img,p_scale=.01,a_std_back)
        series_e[i] = np.sqrt(centered_result_P['g1']**2 + centered_result_P['g2']**2)
    stacked_ellipticities.append(series_e)
print stacked_ellipticities

In [ ]:
pts = [60./(2**i) for i in range(1,11)]
ell = [np.sum(i)/len(i) for i in stacked_ellipticities]
err = [np.var(i) for i in stacked_ellipticities]
print err
plt.figure(figsize=(10,4))
plt.title('ellipticity after centering',fontsize=12)
plt.ylabel('g',fontsize=12)
plt.xlabel('length of exposure',fontsize=12)
plt.errorbar(pts,ell,fmt='o',yerr=err)
plt.xscale('log')
plt.show()

### starting small

In [ ]:
N = 10
p_scale = 0.01
centered_imgs = []
deltas = np.zeros((2,N))
for i in range(N):
    result_P = fit_single_exposure(dssi_a_050[i],p_scale,wl)
    cx,cy = result_P['offset_x'],result_P['offset_y']
    centered_imgs.append( dssi_a_050[i][:256+2*int(cy),2*int(cx):] )    
    deltas[:,i] = [result_P['g1'],result_P['g2']]
print deltas

In [ ]:
## find smallest images, and clip so that they're all the same size
imgShapes = [np.shape(centered_imgs[i]) for i in range(N)]
ymin, xmin = np.min(imgShapes, axis=0)
newShape = (ymin,xmin)

shape_deltas = [(a-ymin,b-xmin) for (a,b) in imgShapes]

trimmed_centered_imgs = np.zeros((N,ymin,xmin))
tc_added = np.zeros((N,ymin,xmin))
for i in range(N):
    (dy,dx) = shape_deltas[i]
    trimmed_centered_imgs[i] = centered_imgs[i][dy/2:imgShapes[i][0]-dy/2,dx/2:imgShapes[i][1]-dx/2]
    tc_added[i] = np.sum(trimmed_centered_imgs,axis=0)

    ## refit and compute difference in fit from 
    result_P_2 = fit_single_exposure(trimmed_centered_imgs[i],p_scale,wl)
    print result_P_2['g1']
    deltas[:,i] -= [(deltas[0,i] - result_P_2['g1']),(deltas[1,i] - result_P_2['g2'])]

In [ ]:
pts = np.linspace(0,N-1,N)
plt.figure(figsize=(10, 6))
plt.plot(pts,deltas[0],'o',label='g1')
plt.plot(pts,deltas[1],'ro',label='g2')
plt.legend()
plt.xlabel('image index')
plt.ylabel('% change in ellipticity', fontsize=12)
plt.show()